In [16]:
#!/usr/bin/env python
# coding: utf-8

# data preparation

import os
import pandas as pd
from glob import glob
import numpy as np


from tensorflow.keras import backend
from tensorflow.keras.models import Sequential
from tensorflow.keras import layers
from keras import models
from keras.layers.advanced_activations import LeakyReLU

from tensorflow.keras.optimizers import Adam
import keras.backend as K
import librosa
import librosa.display
import pylab
import matplotlib.pyplot as plt
from matplotlib import figure
import gc
from path import Path


from keras_preprocessing.image import ImageDataGenerator
train_datagen = ImageDataGenerator(rescale=1.0/255)
test_datagen = ImageDataGenerator(rescale=1.0/255)

train_generator = train_datagen.flow_from_directory(
        r'D:\reposetory\Save_Transport\dataset\train',
        target_size=(150, 150),
        batch_size=20)

valik = test_datagen.flow_from_directory(
        r'D:\reposetory\Save_Transport\dataset\test',
        target_size=(150, 150),
        batch_size=20
        )


Found 1696 images belonging to 2 classes.
Found 60 images belonging to 2 classes.


In [3]:
#!/usr/bin/env python
# coding: utf-8

# model


import tensorflow as tf
import keras.backend as K
def weighted_binary_crossentropy(y_true, y_pred):
    weights = (tf.math.abs(y_true-1) * 59.) + 1.
    bce = tf.keras.losses.binary_crossentropy(y_true, y_pred)
    weighted_bce = K.mean(bce * weights)
    return weighted_bce

model = tf.keras.models.Sequential([
    tf.keras.layers.Conv2D(16, (3, 3), input_shape=(150, 150, 3)),
    tf.keras.layers.Conv2D(16, (3, 3)),
    tf.keras.layers.MaxPooling2D(2, 2),
    tf.keras.layers.LeakyReLU(alpha=0.3),
    tf.keras.layers.Conv2D(32, (3, 3)),
    tf.keras.layers.Conv2D(32, (3, 3)),
    tf.keras.layers.MaxPooling2D(2, 2),
    tf.keras.layers.LeakyReLU(alpha=0.3),
    tf.keras.layers.Conv2D(64, (3, 3)),
    tf.keras.layers.Conv2D(64, (3, 3)),
    tf.keras.layers.MaxPooling2D(2, 2),
    tf.keras.layers.LeakyReLU(alpha=0.3),
    tf.keras.layers.Flatten(),
    tf.keras.layers.Dense(512),
    tf.keras.layers.LeakyReLU(alpha=0.3),
    tf.keras.layers.Dense(2, activation='sigmoid')
])


# Настраиваем гиперпараметры нейронной сети
model.compile(optimizer='adam', loss='binary_crossentropy', metrics=['accuracy'])

# Начинаем обучение
history = model.fit(train_generator,
                      validation_data=valik,
                      steps_per_epoch=24,
                      epochs=10,
                      validation_steps=3,
                      verbose=2)

model.save('save_1.h5')



Epoch 1/10
24/24 - 8s - loss: 0.4742 - accuracy: 0.7917 - val_loss: 0.4762 - val_accuracy: 0.6833
Epoch 2/10
24/24 - 9s - loss: 0.2950 - accuracy: 0.8250 - val_loss: 0.3209 - val_accuracy: 0.9000
Epoch 3/10
24/24 - 8s - loss: 0.2511 - accuracy: 0.8562 - val_loss: 1.0799 - val_accuracy: 0.5000
Epoch 4/10
24/24 - 8s - loss: 0.2992 - accuracy: 0.8403 - val_loss: 0.3363 - val_accuracy: 0.8500
Epoch 5/10
24/24 - 8s - loss: 0.2621 - accuracy: 0.8592 - val_loss: 0.3290 - val_accuracy: 0.9167
Epoch 6/10
24/24 - 8s - loss: 0.2153 - accuracy: 0.8571 - val_loss: 0.3923 - val_accuracy: 0.7500
Epoch 7/10
24/24 - 9s - loss: 0.2542 - accuracy: 0.8354 - val_loss: 0.2572 - val_accuracy: 1.0000
Epoch 8/10
24/24 - 8s - loss: 0.2659 - accuracy: 0.8313 - val_loss: 0.3284 - val_accuracy: 0.8500
Epoch 9/10
24/24 - 9s - loss: 0.2572 - accuracy: 0.8256 - val_loss: 0.4488 - val_accuracy: 0.5000
Epoch 10/10
24/24 - 8s - loss: 0.2163 - accuracy: 0.8750 - val_loss: 0.2144 - val_accuracy: 1.0000


In [24]:
from warnings import ignore
warnings.ignore()
from tensorflow.keras.models import load_model
import numpy as np
import matplotlib.pyplot as plt
#from data_prep import valik
import pandas as pd
from sklearn.metrics import confusion_matrix
from PIL import Image, ImageFont, ImageDraw


model = load_model(r'D:\reposetory\Save_Transport\main_classifier\save_1.h5')


results = model.evaluate(valik)
print('\n loss, accuracy =', results, '\n')


final_res = model.predict(valik[1][0])


print(final_res)




data = {'y_Actual':    [ tuple(i) for i in valik[1][1]],
        'y_Predicted': [ tuple(i) for i in final_res]
        }

df = pd.DataFrame(data, columns=['y_Actual','y_Predicted'])
print (df)


ImportError: cannot import name 'ignore' from 'warnings' (C:\Users\Alexe\anaconda3\envs\audioClsfy\lib\warnings.py)